In [1]:
import json
import urllib.parse
import requests
import numpy as np
import pandas as pd


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
full_df = pd.DataFrame()

#set the column types for the data frames we are going to load
column_types={'complaint_type': np.unicode,
   'incident_zip': np.unicode,
  }

offset = 0
chunk_size = 100000
while True:
    #retrieve data in chunks using json api as it lets us filter out years and select only certain columms
    url_base = 'https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$limit=%s&$offset=%s' % (chunk_size,offset)
    select_clause = '$select=:id,created_date,complaint_type,incident_zip,borough'
    #create a filter to get only 2017 data
    where_clause = '$where=' + urllib.parse.quote("created_date between '2017-01-01T00:00:00' and '2017-12-31T23:59:59'")
    #create the full url
    url = '%s&%s&%s' % (url_base,select_clause,where_clause)

    #now get the filtered data
    result = requests.get(url)

    #load the data in to a data frame and append it to the full one
    current_df = pd.read_json(result.text, dtype=column_types)
    data_read = current_df.shape[0]

    #remove rows with unspecified boroughs
    current_df = current_df[current_df.borough != 'Unspecified']
    #now add to the full one
    full_df = pd.concat([full_df,current_df])

    #now increase the offset and exit if we read less than the full amount
    offset += chunk_size
    if data_read < chunk_size:
        break




In [3]:
#Problem 1
#Consider only the 10 most common overall complaint types. For each borough, how many of each of those 10 types were there in 2017?

#get the top 10 complaints by type in a list
complaint_counts = full_df.groupby(['complaint_type'])[':id'].agg('count')
top_complaints = complaint_counts.nlargest(10).to_dict().keys()

#now use the top 10 complaints to filter the main list
filtered_top_complaint_df = full_df[full_df['complaint_type'].isin(top_complaints)]
#now get the top complaints per borough
borough_top_complaint_df = filtered_top_complaint_df.groupby(['borough','complaint_type'])[':id'].agg('count')
borough_top_complaint_df



borough        complaint_type         
BRONX          Blocked Driveway           23811
               HEAT/HOT WATER             74587
               Illegal Parking            15685
               Noise                       3134
               Noise - Residential        55531
               Noise - Street/Sidewalk    13972
               Street Condition           11972
               Street Light Condition     20616
               UNSANITARY CONDITION       24812
               Water System               10509
BROOKLYN       Blocked Driveway           47869
               HEAT/HOT WATER             73361
               Illegal Parking            53807
               Noise                      15547
               Noise - Residential        65816
               Noise - Street/Sidewalk    21187
               Street Condition           25068
               Street Light Condition     24605
               UNSANITARY CONDITION       27384
               Water System               18500
M

In [4]:
#Problem 2
#For the 10 most populous zip codes, how many of each of those 10 types were there in 2017?

#load a data frame with zipcode population information from the census
zipcode_column_types={'Zip Code ZCTA': np.unicode }
zipcode_population_df = pd.read_csv('2010_Census_Population_By_Zipcode_ZCTA.csv',dtype=zipcode_column_types).rename(
                     columns={'Zip Code ZCTA': 'incident_zip','2010 Census Population':'Zip Population'})

#get distinct nyc zipcodes (might want to replace the with a separate imported dataset)
zipcodes = full_df.incident_zip.unique()

#filter the population data set by the distinct zipcodes to just get nyc populations
nyc_zipcode_population_df = zipcode_population_df[zipcode_population_df['incident_zip'].isin(zipcodes)]
#print nyc_zipcode_population_df.head()

#now get the top ten most populous zipcodes from nyc (this might need to be improved to be done outset our complaint set)
ten_largest_zipcodes_df = nyc_zipcode_population_df.nlargest(10, 'Zip Population')#.to_dict().keys()

#now merge the top ten zipcodes set with the top 10 complaint set we previously calculated for problem 1
complaints_in_top_zipcodes_df = ten_largest_zipcodes_df.merge(filtered_top_complaint_df[['complaint_type','incident_zip']], on='incident_zip')
#now get the aggegate count of complaint type for each of the 10 zipcodes
complaints_in_top_zipcodes_agg_df = complaints_in_top_zipcodes_df.groupby(['incident_zip', 'complaint_type']).count()
complaints_in_top_zipcodes_agg_df



Zip Population
incident_zip complaint_type                         
10025        Blocked Driveway                    129
             HEAT/HOT WATER                     2317
             Illegal Parking                     712
             Noise                              1414
             Noise - Residential                2035
             Noise - Street/Sidewalk            1198
             Street Condition                    636
             Street Light Condition               22
             UNSANITARY CONDITION                638
             Water System                        461
10467        Blocked Driveway                   2002
             HEAT/HOT WATER                     6753
             Illegal Parking                     944
             Noise                               280
             Noise - Residential                5633
             Noise - Street/Sidewalk             712
             Street Condition                    594
             Street Light Condition              455
             UNSANITARY CONDITION               2221
             Water System                        576
11207        Blocked Driveway                   2034
             HEAT/HOT WATER                     2647
             Illegal Parking                    1448
             Noise                               195
             Noise - Residential                2981
             Noise - Street/Sidewalk             550
             Street Condition                   1103
             Street Light Condition              993
             UNSANITARY CONDITION               1644
             Water System                        776
...                                              ...
11368        Blocked Driveway                   4203
             HEAT/HOT WATER                     1759
             Illegal Parking                    1207
             Noise                               150
             Noise - Residential                2413
             Noise - Street/Sidewalk             674
             Street Condition                    555
             Street Light Condition              498
             UNSANITARY CONDITION                629
             Water System                        608
11373        Blocked Driveway                   2522
             HEAT/HOT WATER                     4202
             Illegal Parking                    1223
             Noise                               279
             Noise - Residential                1754
             Noise - Street/Sidewalk             307
             Street Condition                    696
             Street Light Condition              396
             UNSANITARY CONDITION                599
             Water System                        362
11385        Blocked Driveway                   2968
             HEAT/HOT WATER                     1837
             Illegal Parking                    3959
             Noise                               522
             Noise - Residential                2508
             Noise - Street/Sidewalk             596
             Street Condition                   1207
             Street Light Condition              850
             UNSANITARY CONDITION                681
             Water System                       1215

[100 rows x 1 columns]

In [6]:
#Problem 3
#Considering all complaint types. Which boroughs are the biggest "complainers" relative to the size of the population in 2017?
#Meaning, calculate a complaint-index that adjusts for population of the borough.

#get borough population information for 2010 from a previously downloaded file
borough_pop_df = pd.read_csv('New_York_City_Population_By_Neighborhood_Tabulation_Areas.csv')
borough_pop_df = borough_pop_df[borough_pop_df['Year'] == 2010]

#convert the boroughs to uppercase
#borough_pop_df['Borough'] = map(lambda x: x.upper(), borough_pop_df['Borough'])
borough_pop_df['Borough'] = borough_pop_df['Borough'].str.upper()
#now aggregate across all areas of each borough to get total population per borough
aggregate_borough_pop_df = borough_pop_df.groupby(['Borough'])['Population'].sum().reset_index()

#count the complaints per borough from the full list of complaints in to a new dataframe
aggregate_borough_complaint_df = full_df.groupby(['borough'])[':id'].count().reset_index().rename(columns={'borough': 'Borough', ':id':'Complaints'})
#now merge the borough complaint set with the population set so we have both at once
full_borough_complaint_df = aggregate_borough_pop_df.merge(aggregate_borough_complaint_df[['Complaints','Borough']], on='Borough')
#get total population
population = full_borough_complaint_df['Population'].sum()
#get total complaints
total_complaints = full_borough_complaint_df['Complaints'].sum()
#calculate complaints per person in each borough
full_borough_complaint_df['Complaints_PP'] = full_borough_complaint_df['Complaints'] / full_borough_complaint_df['Population']
#calculate complaint percentage divided by population percentage
full_borough_complaint_df['Complaint_Index'] = (full_borough_complaint_df['Complaints'] / total_complaints) / (full_borough_complaint_df['Population'] / population)
full_borough_complaint_df

,Borough,Population,Complaints,Complaints_PP,Complaint_Index
0,BRONX,1385108,454744,0.328309,1.116639
1,BROOKLYN,2504700,764476,0.305217,1.038096
2,MANHATTAN,1585873,470603,0.296747,1.009290
3,QUEENS,2230722,588197,0.263680,0.896823
4,STATEN ISLAND,468730,125597,0.267952,0.911352
